In [1]:
# Module Importations
from scipy.stats import reciprocal
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import keras
import numpy as np
import time

print('keras.__version__=', keras.__version__)

Using TensorFlow backend.
keras.__version__= 2.3.1


In [2]:
# Project Module Importations
from src.data import load_data
from src.data import split_data
from src.models import keras_helpers

In [3]:
# Load dataset
original_dataset = load_data.load_motor_data()

In [4]:
# Split data into training / evaluation sets
training_set, evaluation_set = split_data.split_train_test(original_dataset, 0.8)

In [5]:
# Drop profile id data column
training_set = training_set.drop("profile_id", axis = 1)
evaluation_set = evaluation_set.drop("profile_id", axis = 1)

In [6]:
# Create rotor target
rotor_training_data = training_set.drop("pm", axis = 1)
rotor_label_data = training_set["pm"].copy()

rotor_test_data = evaluation_set.drop("pm", axis = 1)
rotor_test_label = evaluation_set["pm"].copy()

In [7]:
# Create train and test arrays
X_train, X_test, y_train, y_test = train_test_split(rotor_training_data, rotor_label_data, test_size = 0.2, random_state = 0)

In [8]:
# Multilayer Perceptron (Target - Rotor Temperature)

# Clear existing models
keras.backend.clear_session()

# Parameter distribution
param_distribs = { 
    "n_hidden": [1, 2, 4], 
    "n_neurons": np.arange(1, 100), 
    "learning_rate": reciprocal(3e-4, 3e-2)
    }

# Build model
wrapped_model = keras_helpers.wrap_model()

# Initialise Random Search
rnd_search_cv = RandomizedSearchCV(wrapped_model, param_distribs, n_iter = 10, cv = 3)

# Name model file
model_id = keras_helpers.name_model()
filepath_full = keras_helpers.make_save_string(model_id)

# Set save & earlystop callbacks
earlystop_cb = keras.callbacks.EarlyStopping(patience = 5)
checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath = filepath_full, save_best_only = True)

# Train model
rnd_search_cv.fit( X_train, y_train, epochs = 50, validation_data =(X_test, y_test), callbacks =[checkpoint_cb, earlystop_cb])

06461 [==============================] - 3s 33us/step - loss: 0.1675 - val_loss: 0.1660
Epoch 8/50
106461/106461 [==============================] - 3s 33us/step - loss: 0.1620 - val_loss: 0.1609
Epoch 9/50
106461/106461 [==============================] - 3s 32us/step - loss: 0.1581 - val_loss: 0.1592
Epoch 10/50
106461/106461 [==============================] - 3s 33us/step - loss: 0.1555 - val_loss: 0.1561
Epoch 11/50
106461/106461 [==============================] - 3s 32us/step - loss: 0.1532 - val_loss: 0.1533
Epoch 12/50
106461/106461 [==============================] - 3s 33us/step - loss: 0.1512 - val_loss: 0.1510
Epoch 13/50
106461/106461 [==============================] - 4s 34us/step - loss: 0.1497 - val_loss: 0.1497
Epoch 14/50
106461/106461 [==============================] - 4s 35us/step - loss: 0.1484 - val_loss: 0.1487
Epoch 15/50
106461/106461 [==============================] - 4s 35us/step - loss: 0.1473 - val_loss: 0.1479
Epoch 16/50
106461/106461 [=======================

RuntimeError: Cannot clone object <keras.wrappers.scikit_learn.KerasRegressor object at 0x0000029D7D3C3E80>, as the constructor either does not set or modifies parameter learning_rate

In [9]:
# Load model
filepath = r'C:\Developer\electric_motor_thermal_modelling\Models\YC_model_2020_07_23-12_37_34.h5'

model = keras.models.load_model(filepath)

model.evaluate(rotor_test_data, rotor_test_label)

798456/798456 [==============================] - 10s 12us/step


0.11478357165518864